In [ ]:
from model import *
from keras.utils import normalize
import os 
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from keras.optimizers import Adam

In [ ]:
import tifffile as tiff
from patchify import patchify, unpatchify 

In [ ]:
large_image_stack=tiff.imread()
large_mask_stack=tiff.imread()

In [ ]:
all_img_patches=[]
for img in range(large_image_stack.shape[0]):
    large_image=large_image_stack[img]
    patches_mask=patchify(large_image,(256,256),step=256)

    for i in range(patches_mask.shape[0]):
        for j in range(patches_mask.shape[1]):

            single_patch_img= patches_mask[i,j,:,:]
            single_patch_img=single_patch_img.astype('float32')/255

            all_img_patches.append(single_patch_img)

images=np.array(all_img_patches)
images=np.expand_dims(images,-1)

In [ ]:
all_mask_patches=[]
for mask in range(large_mask_stack.shape[0]):
    large_mask=large_mask_stack[mask]
    patches_mask=patchify(large_mask,(256,256),step=256)

    for i in range(patches_mask.shape[0]):
        for j in range(patches_mask.shape[1]):

            single_patch_mask= patches_mask[i,j,:,:]
            single_patch_mask=single_patch_mask.astype('float32')/255

            all_mask_patches.append(single_patch_mask)

masks=np.array(all_mask_patches)
masks=np.expand_dims(mask s,-1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(images,masks,test_size=0.2)

In [ ]:
IMG_HEIGHT=images.shape[1]
IMG_WIDTH=images.shape[2]
IMG_CHANNELS=images.shape[3]

In [ ]:
input_shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
model=build_unet(input_shape)
model.compile(optimizer=Adam(lr=1e-3),loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

In [ ]:
seed=24 
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_data_gen_args=dict(rotation_range=90,
                       width_shift_range=0.3,
                       shear_range=0.5,
                       zoom_range=0.3
                       horizantal_flip=True,
                       vertical_flip=True,
                       fill_mode="reflect")

maskdata_gen_args=dict(rotation_range=90,
                       width_shift_range=0.3,
                       shear_range=0.5,
                       zoom_range=0.3
                       horizantal_flip=True,
                       vertical_flip=True,
                       fill_mode="reflect",
                       preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype)) #Binarize the output again. )

In [ ]:

image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_data_generator.fit(x_train, augment=True, seed=seed)

image_generator = image_data_generator.flow(x_train, seed=seed)
valid_img_generator = image_data_generator.flow(x_test, seed=seed)

mask_data_generator = ImageDataGenerator()
mask_data_generator.fit(y_train, augment=True, seed=seed)
mask_generator = mask_data_generator.flow(y_train, seed=seed)
valid_mask_generator = mask_data_generator.flow(y_test, seed=seed)

In [ ]:

def my_image_mask_generator(image_generator, mask_generator):
    train_generator = zip(image_generator, mask_generator)
    for (img, mask) in train_generator:
        yield (img, mask)

my_generator = my_image_mask_generator(image_generator, mask_generator)

validation_datagen = my_image_mask_generator(valid_img_generator, valid_mask_generator)


In [ ]:
x = image_generator.next()
y = mask_generator.next()
for i in range(0,1):
    image = x[i]
    mask = y[i]
    plt.subplot(1,2,1)
    plt.imshow(image[:,:,0], cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(mask[:,:,0])
    plt.show()

In [ ]:
batch_size = 16
steps_per_epoch = 3*(len(x_train))//batch_size
history = model.fit(my_generator, validation_data=validation_datagen, steps_per_epoch=steps_per_epoch, validation_steps=steps_per_epoch, epochs=100)

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
#IOU
import random 
y_pred=model.predict(x_test)
y_pred_thresholded = y_pred > 0.5

intersection = np.logical_and(y_test, y_pred_thresholded)
union = np.logical_or(y_test, y_pred_thresholded)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU socre is: ", iou_score)

#Predict on a few images
#model = get_model()
#model.load_weights('mitochondria_50_plus_100_epochs.hdf5') #Trained for 50 epochs and then additional 100

test_img_number = random.randint(0, len(x_test))
test_img = x_test[test_img_number]
ground_truth=y_test[test_img_number]
test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)
prediction = (model.predict(test_img_input)[0,:,:,0] > 0.2).astype(np.uint8)

plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(prediction, cmap='gray')

plt.show()